# Type Investigation 2 : Calculating Deviances and Balance Values for Type Sets

---

In this notebook I'm going to calculate how far type-sets are from the averages as well as the balance values (a value that I have made up based on previous analysis)

Given the notes from the Type-Investigation Notebook. I have added a quantified number associated with each Advantage and Disadvantage I'll be looking at. 
- Negative Numbers are detractors, and result from being weaker than the average or norm.
- Positive Numbers are promoters, and result from being stronger than the average or norm.
- Zero Values are neutral and indicate that this is representative of the average or norm.

Summing these values in the end will result in overall assessment of these new Pokémon types I'm investigating.

- Sums around 0 are Acceptable and will be recommended.
- Sums much greater than 0 are "too strong" and will most likely be not advised.
- Sums much less than 0 are "too weak" and will most likely be not advised.

In [1]:
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

poke_pivot = pd.read_csv('Data/poke_pivot.csv',       index_col = 0) 
advantages = pd.read_csv('Data/advantages.csv',       index_col = 0)
pokemon    = pd.read_csv('Data/pokedex_formal_2.csv', index_col = 0)

type_recommendations = pickle.load( open( "./Python-Files/type_i.p", "rb" ) )

In [2]:
# same function as before
# Using this function on the Types I wanted to Investigate.

def capabilities(type1, type2, advantages = advantages):  
    # Offensive Advantages
    advantages_o = []
    for types in [type1, type2]:
        try:
            [advantages_o.append(item) for item in list(advantages[advantages.T[types] == 2.0].index.values) if item not in advantages_o]
        except:
            pass    
    # Offensive Disadvantages
    disadvantages_o = []
    for def_type in advantages.columns:
        sub_set = list(advantages.T[[type1, type2]].loc[def_type])
        if (sub_set[0] < 1 and  sub_set[1] < 1):
            max_weak = max(sub_set)
            disadvantages_o.append('%s : %s' %(def_type,max_weak ))       
    # Offensive Power
    offensive_power = sum([a if a > b else b for a,b in advantages.loc[[type1, type2]].T.values])   
    # Defensive Advantages, Disadvantages & Immunities
    advantages_d = []
    disadvantages_d = []
    immunity = []
    defensive_power = 0
    # If statement for similar types
    if type1 == type2:
        defensive_power = advantages[type1].sum()
        for name, value in zip(advantages[['Fire']].index, advantages[['Fire']].values) :
            if value[0] == 0:
                immunity.append(name)   
            if 0 < value[0] < 1 :
                advantages_d.append('%s : %s' %(name, value[0]))   
            if value[0] > 1:
                disadvantages_d.append('%s : %s' %(name, value[0]))       
    else:
        for types in advantages.columns:
            row = advantages[[type1,type2]].loc[types]
            defensive_power += row[0]*row[1]
            if 0 < row[0]*row[1] < 1:
                advantages_d.append('%s : %s' %(row.name, row[0]*row[1]))
            if  row[0]*row[1] > 1:
                disadvantages_d.append('%s : %s' %(row.name, row[0]*row[1]))
            if row[0]*row[1] == 0:
                immunity.append(row.name)    
    # dictionary for storing info 
    capabilitys = {'off_adv'       : advantages_o    , 'count_off_adv' : len(advantages_o),
                    'off_disadv'    : disadvantages_o , 'count_off_dis' : len(disadvantages_o),
                    'off_pwr'       : offensive_power ,
                    'def_adv'       : advantages_d    , 'count_def_adv' : len(advantages_d),
                    'def_disadv'    : disadvantages_d , 'count_def_dis' : len(disadvantages_d),
                    'def_pwr'       : defensive_power ,
                    'immunities'    : immunity        , 'count_imm'     : len(immunity)}   
    return capabilitys

# Same Function to convert output to list
def cap_to_list(capability):
    O_list = [capability['count_def_adv'], capability['count_def_dis'], capability['def_adv'],
              capability['count_off_adv'], capability['count_off_dis'], capability['off_adv'],
              capability['count_imm'    ], capability['def_disadv'   ], capability['def_pwr'], 
              capability['immunities'   ], capability['off_disadv'   ], capability['off_pwr']]
    return O_list

In [3]:
# Setting all the type-set pairs
# utilizing alphabetical order to eliminate redundencies
# i.e  (Dark, Ice), (Ice, Dark)
investigation_pairs = []
for type_name1, type_list in type_recommendations.items():
    for type_name2 in type_list:
        investigation_pairs.append(tuple(sorted([type_name1, type_name2])))
        
investigation_set = set(investigation_pairs)

In [4]:
# Dict column names
new_cols = [ 'type_combo',
            'count_def_adv', 'count_def_dis', 'def_adv', 
            'count_off_adv', 'count_off_dis', 'off_adv',
            'count_imm'    , 'def_disadv'   , 'def_pwr',
            'immunities'   , 'off_disadv'   , 'off_pwr']

#empty dict to fill
inv_type_df = pd.DataFrame(columns = new_cols)
# filling with list info after passing investigation type_set through the capabilities function
for type_set in investigation_set:
    cap_list = cap_to_list(capabilities(type_set[0], type_set[1]))
    cap_list.insert(0, type_set)
    inv_type_df.loc[len(inv_type_df)] = cap_list

In [5]:
inv_type_df.head(2)

,type_combo,count_def_adv,count_def_dis,def_adv,count_off_adv,count_off_dis,off_adv,count_imm,def_disadv,def_pwr,immunities,off_disadv,off_pwr
0,"(Electric, Normal)",3,2,"[Electric : 0.5, Flying : 0.5, Steel : 0.5]",2,0,"[Flying, Water]",1,"[Fighting : 2.0, Ground : 2.0]",17.5,[Ghost],[],20.0
1,"(Dragon, Fairy)",7,4,"[Bug : 0.5, Dark : 0.5, Electric : 0.5, Fighti...",3,1,"[Dragon, Dark, Fighting]",1,"[Fairy : 2.0, Ice : 2.0, Poison : 2.0, Steel :...",17.5,[Dragon],[Steel : 0.5],20.5


In [6]:
focus_stats_summary = pd.read_csv("./Data/focus_stats.csv", index_col = 0)
focus_stats_summary

,count_def_adv,count_def_dis,def_pwr,count_off_adv,count_off_dis,off_pwr,count_imm
count,667.000000,667.000000,667.000000,667.000000,667.000000,667.000000,667.000000
mean,5.308846,3.416792,19.060345,3.850075,2.539730,20.449025,0.440780
std,1.613744,1.071142,1.911326,1.998876,1.961222,2.732648,0.714932
min,0.000000,1.000000,13.250000,0.000000,0.000000,16.000000,0.000000
25%,5.000000,3.000000,18.000000,3.000000,1.000000,18.000000,0.000000
50%,6.000000,3.000000,18.750000,4.000000,3.000000,20.000000,0.000000
75%,6.000000,4.000000,20.000000,5.000000,3.000000,23.000000,1.000000
max,11.000000,7.000000,26.000000,9.000000,7.000000,27.000000,3.000000


In [7]:
focus_stats_summary.loc[['mean','std']]

,count_def_adv,count_def_dis,def_pwr,count_off_adv,count_off_dis,off_pwr,count_imm
mean,5.308846,3.416792,19.060345,3.850075,2.539730,20.449025,0.440780
std,1.613744,1.071142,1.911326,1.998876,1.961222,2.732648,0.714932


**After looking at the Mean, Median and Mode as well as the distributions from the Type-Investigation Notebooks, I have determined that they will be the neutral values for each attribute of focus.**


|Attribute | Measure of Central Tendency | Value |
|:----|:----:|----:|
|Count Defensive Advantages    | Median/Mode | 6 |
|Count Defensive Disadvantages | Median/Mode | 3 |
|Defensive Power               | Mean        | 19|
|Count Offensive Advantages    | Median/Mean | 4 |
| Count Offensive Disadvantages | Median/Mode | 3|
| Offensive Power               | Median/Mean | 20|
| Count of Immunities           | Mode        | 0|

**Balance Value Metric**
After analyzing the centrality of the distributions in the previous notebook I identified what I believe to be the best measure of central tendency for each of the stats I'm looking into.  

The Balance Value Metric is my 8th engineered feature for this analysis and is the combination of the previous 7 engineered features.  Positive and negative values are assigned to each of the attributes of focus based on how far away the calculate value for the potential type-set is from the point of centrality I have identified. These values are not consistent and instead rely heavily on how I interpreted the histograms on the previous page.  

- A sum of these values that is very large is indicative of a type-set that is consistently above the central value (better than average)
- A sum of these that is close to zero is indicative of a type-set that is a balance recommendation
- A sum of these values that is very small(negative) is indicative of a type-set that is consistently below the central value (worse than average)

In [8]:
# Count Defensive Advantages
def defensive_adv_cal(dav):
    if dav   <= 1: # 0,1    : -3
        return -5
    elif dav <= 3: # 2,3    : -2
        return -2
    elif dav <= 5: # 4,5    : -1
        return -1
    elif dav == 6: # 6      : 0
        return 0
    elif dav <= 7: # 7      : +1
        return 1
    else:          # 8,9,10 : +3  
        return 3

# Count Defensive Disadvantages
def defensive_dis_cal(ddv):
    if ddv  <=  2: # 1,2     : +2
        return 2
    elif ddv == 3: # 3       :  0
        return 0
    else:          # 4,5,6,7 : -1
        return -1

# Count of Offensive Advantages
def offensive_adv_cal(oav):
    if oav   <= 1:# 1     : -1
        return -1
    elif oav <= 4:# 2,3,4 :  0
        return 0
    elif oav <= 6:# 5,6   : +1
        return 1
    else:         # 7, 8  : +2
        return 2

# Count of Offensive Disadvantages
def offensive_dis_cal(odv):
    if odv    < 1:# 0       : +1
        return 1
    elif odv <= 4:# 1,2,3,4 :  0
        return 0
    else:         # 5, 6    : -3
        return -3

# Defensive Power
def defensive_pow_cal(dpv):
    if dpv   <= 15:#  < 15   : +2
        return 2
    elif dpv <= 17:# 15 - 17 : +1
        return 1
    elif dpv <= 21:# 17 - 21 :  0
        return 0
    elif dpv <= 23:# 21 - 23 : -1
        return -1
    else:          # > 23    : -2
        return -2

# Offensive Power
def offensive_pow_cal(opv):
    if opv    <=  18:# 15.5 - 18 : -1
        return -1
    elif opv <=   23:# 18 - 23   :  0
        return 0
    elif opv <= 25.5:# 23 - 25.5 : +1
        return 1
    else:            # > 25.5    : +2
        return 2

# Count of Immunites
def immunity_cal(civ):
    if   civ ==0:# 0 :  0
        return 0
    elif civ ==1:# 1 : +1
        return 1
    elif civ ==2:# 2 : +2
        return 2
    else:        # 3 : +3
        return 3





In [9]:
# While Standard deviation is used, this is not standardized given the 
#CT (Central Tendency) could be Mean, Median or Mode
def deviation(value,CT, std):
    return abs(value-CT)/std

In [10]:
# testing the equation
deviation(7, 10,5)

0.6

In [11]:
#investigation_set
invest_dict = {'{} & {}'.format(i_set[0],i_set[1]) : capabilities(i_set[0],i_set[1])  for i_set in investigation_set   }


In [12]:
# Central Values and Distributions for each Focus Attribute
fss_dict = focus_stats_summary.loc[['mean','std']].to_dict()
fss_dict

{'count_def_adv': {'mean': 5.308845577211394, 'std': 1.6137443687056452},
 'count_def_dis': {'mean': 3.4167916041979014, 'std': 1.0711420290025904},
 'count_imm': {'mean': 0.4407796101949025, 'std': 0.7149316885012674},
 'count_off_adv': {'mean': 3.850074962518741, 'std': 1.9988763721473448},
 'count_off_dis': {'mean': 2.5397301349325336, 'std': 1.9612220984008133},
 'def_pwr': {'mean': 19.060344827586206, 'std': 1.9113257433340884},
 'off_pwr': {'mean': 20.44902548725637, 'std': 2.732648471266951}}

In [13]:
# Calculate difference for each type-set from the central points
attributes = ('count_def_adv', 'count_def_dis', 'count_off_adv', 'count_off_dis','count_imm', 'def_pwr', 'off_pwr')



for type_set_key in invest_dict.keys():
    # Balance Value Calculation
    cs = invest_dict[type_set_key]
    balance_calc = sum([defensive_adv_cal(cs['count_def_adv']), defensive_dis_cal(cs['count_def_dis']),
                        offensive_adv_cal(cs['count_off_adv']), offensive_dis_cal(cs['count_off_dis']),
                        defensive_pow_cal(cs['def_pwr'])      , offensive_pow_cal(cs['off_pwr']),
                        immunity_cal(cs['count_imm'])])
    invest_dict[type_set_key]['balance_value'] = balance_calc
    
    # Individual deviation calculation
    deviation_list = []
    for value_name in attributes:
        
        deviation_list.append(deviation(invest_dict[type_set_key][value_name],
                  fss_dict[value_name]['mean'],
                  fss_dict[value_name]['std']))
    
    invest_dict[type_set_key]['deviation_list'] = deviation_list
    
    
    


In [14]:
print(invest_dict['Bug & Dark']['balance_value'])
print(invest_dict['Bug & Dark']['deviation_list'])

-1
[0.8110612824391729, 1.4780564602402226, 0.07500465740169723, 0.27520092465439344, 0.7822011512420268, 0.49162481889386, 0.20162656065607223]


**I want this same information for existing types as a comparison.**

In [15]:
pokemon3 = pd.read_csv('./Data/pokemon3', index_col = 0) # read csv

existing_types = [eval(typeCombo) for typeCombo in set(pokemon3['Type'].tolist())] # extrac unique typings
existing_types = set([tuple(sorted(item)) for item in existing_types])
existing_capabilities = {} # convert typing info to dict

In [16]:
# Running Existing Type sets through the Capabilities Function
# ANd adding them to my existin_capabilites Dict
for i_set in existing_types:
    if len(i_set) == 1:
        existing_capabilities[i_set[0]] = capabilities(i_set[0],i_set[0])
    else:
        existing_capabilities['{} & {}'.format(i_set[0],i_set[1])] = capabilities(i_set[0],i_set[1])
        
# Getting Balance Value and Deviation list for knowns
for type_set_key in existing_capabilities.keys():
    cs = existing_capabilities[type_set_key]
    balance_calc = sum([defensive_adv_cal(cs['count_def_adv']), defensive_dis_cal(cs['count_def_dis']),
                        offensive_adv_cal(cs['count_off_adv']), offensive_dis_cal(cs['count_off_dis']),
                        defensive_pow_cal(cs['def_pwr'])      , offensive_pow_cal(cs['off_pwr']),
                        immunity_cal(cs['count_imm'])])
    existing_capabilities[type_set_key]['balance_value'] = balance_calc
    # Get the deviation information for each existin type set
    deviation_list = []
    for value_name in attributes:
        
        deviation_list.append(deviation(existing_capabilities[type_set_key][value_name],
                  fss_dict[value_name]['mean'],
                  fss_dict[value_name]['std']))
    
    existing_capabilities[type_set_key]['deviation_list'] = deviation_list

In [17]:
# I've Done it!

In [18]:
import pickle
pickle.dump( invest_dict, open( "./Python-Files/investigation_dict.p", "wb" ) )
pickle.dump( existing_capabilities, open( "./Python-Files/existing_dict.p", "wb" ) )